In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool,PDFSearchTool
from crewai import LLM


In [ ]:

search_tool = SerperDevTool(n_results=3)

In [ ]:


# Create an agent with all available parameters
agent = Agent(
    role="Topic Analysist",
    goal="Using the tool that i gave to you analyze and interpret complex datasets to provide actionable insights",
    backstory="With over 10 years of experience in data science and machine learning, "
              "you excel at finding patterns in complex datasets.",
    llm=ollama_llm, 
    tools=[SerperDevTool()],
    verbose=True
)

In [ ]:
result = agent.kickoff("Chi è Kenan Yildiz")

print(result.raw)

 Provare in autonomia a creare una crew con un solo agente ed un solo task, con crewAI in un file .py/.ipynb con la configurazione che si preferisce, in modo che esegua la ricerca a piacere su di un file PDF (https://www.its-ictacademy.com/wp-content/uploads/2024/10/Regolamento-Iscrizioni-1.66fd1de210a445.09813304.pdf) tramite il tool PDFSearchTool configurato con llm ed embedder locale.


In [ ]:
ollama_llm= LLM(model="ollama/qwen3:8b",base_url="http://localhost:11434")



In [ ]:
tool = PDFSearchTool(
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="bge-m3:latest",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/bge-m3:latest",
                # title="Embeddings",
            ),
        ),
    )
)

## Es 2


In [ ]:
from crewai import Agent, Crew, Task, Process

agent_one = Agent (
            role="Researcher",
            goal="Synopsis specialist, expert, actors and details about the {film}",
            backstory="A diligent researcher with a keen eye for detail",
            llm=ollama_llm,
            verbose=True,
            tools=[SerperDevTool()]
        )

agent_two = Agent(
            role="Cinematographic critique",
            goal="Expert about the analysis and rewiew about {film}",
            backstory="A diligent researcher with a keen eye for detail",
            llm=ollama_llm,
            verbose=True
        )

task_1 = Task(
            description="Search the informations about the movie, informations about actors, and very deep details and trivia's about the movie",
            expected_output="A text with the summarized informations obtained in the research",
            agent=agent_one
        )

task_2 = Task(
            description="Use the output of the Researcher to create a review about the movie in question",
            expected_output="A text with the review of the film, analyzed in details and with the focus on what could be improved",
            agent=agent_two
        )




In [ ]:
crew = Crew(
            agents=[agent_one, agent_two],
            tasks=[task_1, task_2],
            process=Process.sequential,
            verbose=True
        )
    

In [ ]:
input_films = {"film":"Inception"}

result = crew.kickoff(inputs=input_films)

print(result.raw)

## ES 3


In [ ]:
ollama_llm= LLM(model="ollama/qwen3:8b",base_url="http://localhost:11434")

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    llm=ollama_llm,
    allow_code_execution=False,
)

# Define your agents
developer = Agent(
    role="Developer",
    goal="Writing an efficent and computational efficent code line, without ",
    backstory="You're an expert programmer, specialized in various programming language.",
    llm=ollama_llm,
    allow_code_execution=False
)

tester = Agent(
    role="Tester",
    goal="identifying the errors in the code line,without letting the code go in error.",
    backstory="You are an old programmer, experienced in your task. You are one of the best code reviewers in the company",
    llm=ollama_llm,
    allow_code_execution=False
)

# Define your task
task_1 = Task(
    description="delegating the other agents in writing the code task in an efficent way",
    expected_output="a full workflow with the other agents,and also a full line of code without any error",
    agent=manager   
)

task_2 = Task(
    description="Writing the {code} at the given input",
    expected_output="some full code lines written by the agents, without any errors. The line of codes should run without any interruption",
    agent=developer   
)


task_3= Task(
    description="the agent should review the code from the developer agent to search for any errors and fix them",
    expected_output="it should return a full line of code without any errors, and some advices to increase the performances",
    agent=tester
)





# Instantiate your crew with a custom manager
crew = Crew(
    agents=[developer, tester],
    tasks=[task_3,task_2,task_1],
    manager_agent=manager,
    process=Process.hierarchical,
    verbose=True
)


In [ ]:
input_code = {"code":"a program that sums the first 10 numbers between 1 an 10"}

result = crew.kickoff(inputs=input_code)

print(result)

## Es Knowledge


In [ ]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

In [ ]:

content1 = "Users name is Andrea. He is 24 years old and lives in Civitavecchia."
string_source = StringKnowledgeSource(content=content1)

In [ ]:
ollama_llm= LLM(model="ollama/bge-m3",base_url="http://localhost:11434")

In [ ]:
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="You are a master at understanding people and their preferences.",
    verbose=True,
    allow_delegation=False,
    llm=ollama_llm,
    embedder={
        "provider":"ollama", "url":"http://localhost:11434/api/embeddings",'model_name':"bge-m3"
    }
)


task = Task(
    description="Answer the following question {question}",
    expected_output="An answer to the question",
    agent=agent
    
)


crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here
)


In [ ]:
result = crew.kickoff(inputs={"question": "What city does Andrea live in and how old is he?"})

## Es 4 da fare
## Es 5 da fare

## Es 6

In [ ]:
from crewai import Agent,Task,Crew
from crewai_tools import ScrapeWebsiteTool
from crewai import LLM,Process


In [32]:
ollama_llm= LLM(model="ollama/qwen3:8b",base_url="http://localhost:11434")
tool = ScrapeWebsiteTool()
tool = ScrapeWebsiteTool(website_url=("https://www.laziodigital.it/artificial-intelligence-and-data-science-specialist/"))


In [ ]:
webagent = Agent(
    role="Web Scraper",
    goal="analyze and extract information requested by the user. The informations are contained in {info} and return them in a clear way without missing any word",
    backstory="You are a professional web scraper, like google you provide informations in the question asked, you are clear and fast",
    llm=ollama_llm,
    tools=[tool],
    embedder={
        "provider":"ollama", "url":"http://localhost:11434/api/embeddings",'model_name':"bge-m3"
    },
    verbose=True,

)

task = Task(
    description="you must analyze the {info} and scrape the website searching around the informations reguarding the requested information",
    expected_output="a clear text information extracted from the website",
    agent=webagent

)

crew = Crew(
    agents=[agent],
    tasks=[task]
    
)





In [35]:
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 565db28e-2796-4cdb-a51b-7cec1321d310                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Scraper                                                                                             │
│                                                                                                                 │
│  Task: you must analyze the {info} and scrape the website searching around the informations reguarding the      │
│  requested information                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\Andrea C\Desktop\its\Py\appia_crewai0177\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\Andrea C\Desktop\its\Py\appia_crewai0177\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Scraper                                                                                             │
│                                                                                                                 │
│  Thought: <think>                                                                                               │
│  Okay, let's tackle this. The user wants me to scrape the website                                               │
│  https://www.laziodigital.it/artificial-intelligence-and-data-science-specialist/ and extract all the           │
│  information related to the requested data. The key here is to return the actual complete content, not a        │
│  summary.                                                                                                       │
│  First, I need to use the Read website content tool. The tool doesn't require any arguments, so the Action      │
│  Input will be an empty JSON object. I'll execute that action to get the content of the page.                   │
│  Once I have the content, I need to make sure I capture everything the user is asking for. Since the user       │
│  mentioned the expected criteria is a clear text with all the information, I should present the entire content  │
│  as the final answer. There's no need to summarize; just return the complete text from the website.             │
│  I should check if there are any specific sections or paragraphs that might be relevant, but since the user     │
│  wants everything, I'll include all the content. Let me make sure there are no missing parts and that the       │
│  final answer is exactly what's retrieved from the site.                                                        │
│  </think>                                                                                                       │
│  Thought: I need to read the content of the specified website to extract the required information.              │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The following text is scraped website content:                                                                 │
│  Artificial Intelligence and Data Science Specialist - LAZIO DIGITAL                                            │
│  Vai al contenuto                                                                                               │
│  HOME                                                                                                           │
│  CHI SIAMO Attiva/disattiva menu                                                                                │
│  La Fondazione                                                                                                  │
│  Mission                                                                                                        │
│  La sede                                                                                                        │
│  CORSI Attiva/disattiva menu                                                                                    │
│  Developer                                                                                                      │
│  Cloud Developer                                                                                                │
│  Game Developer                                                                                                 │
│  Cyber Security Expert                                                                                          │
│  Artificial Intelligence and Data Science Specialist                                                            │
│  Digital Media Specialist                                                                                       │
│  ISCRIVITI                                                                                                      │
│  NEWS Attiva/disattiva menu                                                                                     │
│  Bandi                                                                                                          │
│  TRASPARENZA Attiva/disattiva menu                                                                              │
│  LEGGE 124/07                                                                                                   │
│  D.Lgs. n. 231/2001                                                                                             │
│  PATTO ANTICORRUZIONE                                                                                           │
│  WHISTLEBLOWING                                                                                                 │
│  PNrr                                                                                                           │
│  COLLABORA Attiva/disattiva menu                                                                                │
│  Aziende                                                                                                        │
│  Professionisti                                                                                                 │
│  Scuole                                                                                                         │
│  CONTATTI Attiva/disattiva menu                                                                                 │
│  Dove siamo                                                                                                     │
│  Come raggiungerci                                                                                              │
│  Main Menu                                                                                                      │
│  HOME                                                 

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Scraper                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Artificial Intelligence and Data Science Specialist - LAZIO DIGITAL                                            │
│  Vai al contenuto                                                                                               │
│  HOME                                                                                                           │
│  CHI SIAMO Attiva/disattiva menu                                                                                │
│  La Fondazione                                                                                                  │
│  Mission                                                                                                        │
│  La sede                                                                                                        │
│  CORSI Attiva/disattiva menu                                                                                    │
│  Developer                                                                                                      │
│  Cloud Developer                                                                                                │
│  Game Developer                                                                                                 │
│  Cyber Security Expert                                                                                          │
│  Artificial Intelligence and Data Science Specialist                                                            │
│  Digital Media Specialist                                                                                       │
│  ISCRIVITI                                                                                                      │
│  NEWS Attiva/disattiva menu                                                                                     │
│  Bandi                                                                                                          │
│  TRASPARENZA Attiva/disattiva menu                                                                              │
│  LEGGE 124/07                                                                                                   │
│  D.Lgs. n. 231/2001                                                                                             │
│  PATTO ANTICORRUZIONE                                                                                           │
│  WHISTLEBLOWING                                                                                                 │
│  PNrr                                                                                                           │
│  COLLABORA Attiva/disattiva menu                                                                                │
│  Aziende                                                                                                        │
│  Professionisti                                                                                                 │
│  Scuole                                                                                                         │
│  CONTATTI Attiva/disattiva menu                                                                                 │
│  Dove siamo                                                                                                     │
│  Come raggiungerci                                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a397c0f1-55b0-4469-b00d-e9822ccf83f5                                                                     │
│  Agent: Web Scraper                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 565db28e-2796-4cdb-a51b-7cec1321d310                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Artificial Intelligence and Data Science Specialist - LAZIO DIGITAL                              │
│  Vai al contenuto                                                                                               │
│  HOME                                                                                                           │
│  CHI SIAMO Attiva/disattiva menu                                                                                │
│  La Fondazione                                                                                                  │
│  Mission                                                                                                        │
│  La sede                                                                                                        │
│  CORSI Attiva/disattiva menu                                                                                    │
│  Developer                                                                                                      │
│  Cloud Developer                                                                                                │
│  Game Developer                                                                                                 │
│  Cyber Security Expert                                                                                          │
│  Artificial Intelligence and Data Science Specialist                                                            │
│  Digital Media Specialist                                                                                       │
│  ISCRIVITI                                                                                                      │
│  NEWS Attiva/disattiva menu                                                                                     │
│  Bandi                                                                                                          │
│  TRASPARENZA Attiva/disattiva menu                                                                              │
│  LEGGE 124/07                                                                                                   │
│  D.Lgs. n. 231/2001                                                                                             │
│  PATTO ANTICORRUZIONE                                                                                           │
│  WHISTLEBLOWING                                                                                                 │
│  PNrr                                                                                                           │
│  COLLABORA Attiva/disattiva menu                                                                                │
│  Aziende                                                                                                        │
│  Professionisti                                                                                                 │
│  Scuole                                                                                                         │
│  CONTATTI Attiva/disattiva menu                                                                                 │
│  Dove siamo                                           

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: f7569439-dddb-4f35-8d6f-61530e5601e0                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/f7569439-dddb-4f35-8d6f-61530e5601e0?access_code=TRA │
│ CE-ec08cc7294                                                                                                   │
│ 🔑 Access Code: TRACE-ec08cc7294                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯